In [1]:
# Data processing
import pandas as pd
import numpy as np
from datetime import datetime

#downlaod data from csv
type_list= ['phq','hc','org']
url_list = [f'https://storage.googleapis.com/momask/stock_{x}.csv' for x in type_list]
data_set = [pd.read_csv(url) for url in url_list]


#select the latest data
stock = []
for data in data_set:
    ff = data.parsed_timestamp == data.parsed_timestamp.max()
    stock.append(data[ff])

#Record the latest time
latest_time = str(data[ff].human_parsed_timestamp.unique()[0])
    
#merge dataframes
stock[0].rename(columns={'code':'licno'}, inplace=True)
phq = stock[0].copy()
del phq['parsed_timestamp']
hc = pd.concat([stock[1].licno,stock[1].tolqty_diff,stock[1].human_parsed_timestamp],axis=1)
org = pd.concat([stock[2].licno,stock[2].tolqty_diff,stock[2].human_parsed_timestamp],axis=1)
phq.columns = org.columns
qty = pd.concat([phq,hc,org], axis=0 , sort = False).reset_index(drop=True)


#join tables with the coordinate data
info = pd.read_csv("map_info.csv" ,encoding = 'utf8')
df = pd.concat([info , qty], axis = 1 )
df['unit'] = '個'
df = df.fillna(0) #gov website removed some institutes without any announcement
df['hov_txt'] = df['txt']+ df['tolqty_diff'].astype(int).astype(str) +df['unit']

C:\Users\ACER\Anaconda3\lib\site-packages\pandas\core\frame.py:4223: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)


In [2]:
#pip install dash
#pip install plotly==4.5.0

In [ ]:
import dash                     #(version 1.0.0)
import dash_table
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

import plotly.offline as py     #(version 4.4.1)
import plotly.graph_objs as go

mapbox_access_token =  open(".token").read()
app = dash.Dash(__name__)

blackbold={'color':'black', 'font-weight': 'bold'}

app.layout = html.Div([
#---------------------------------------------------------------
# Map_legen + Borough_checklist + Recycling_type_checklist + Web_link + Map
    html.Div([
        html.Div([
            # Map-legend
            html.Ul([
                html.Li("Pharmacy", className='circle', style={'background': '#0000ff','color':'black',
                    'list-style':'none','text-indent': '17px','white-space':'nowrap'}),
                html.Li("HealthCenter", className='circle', style={'background': '#FF0000','color':'black',
                    'list-style':'none','text-indent': '17px'}),
                html.Li("Organisation", className='circle',  style={'background': '#824100','color':'black',
                    'list-style':'none','text-indent': '17px'}),
            ], style={'border-bottom': 'solid 3px', 'border-color':'#00FC87','padding-top': '6px'}
            ),

            # Borough_checklist
            html.Label(children=['Display: '], style=blackbold),
            dcc.Checklist(id='boro_name',
                    options=[{'label':str(b),'value':b} for b in sorted(df['boro'].unique())],
                    value=[b for b in sorted(df['boro'].unique())],
            ),

            # Recycling_type_checklist
            html.Label(children=['Looking for masks: '], style=blackbold),
            dcc.Checklist(id='recycling_type',
                    options=[{'label':str(b),'value':b} for b in sorted(df['type'].unique())],
                    value=[b for b in sorted(df['type'].unique())],
            ),

            # Web_link
            html.Br(),
            html.Label(['Sources:'],style=blackbold),
            html.Pre(id='web_link', children=[],
            style={'white-space': 'pre-wrap','word-break': 'break-all',
                 'border': '1px solid black','text-align': 'center',
                 'padding': '12px 12px 12px 12px', 'color':'blue',
                 'margin-top': '3px'}
            ),
            
            # Noted
            html.Br(),
            html.Label(['Last Updated Time:'],style=blackbold),
            html.Pre(id='noted', children=[latest_time],
            style={'white-space': 'pre-wrap','word-break': 'break-all',
                 'border': '1px solid black','text-align': 'center',
                 'padding': '12px 12px 12px 12px', 'color':'blue',
                 'margin-top': '3px'}
            ),            
            
            

        ], className='three columns'
        ),

        # Map
        html.Div([
            dcc.Graph(id='graph', config={'displayModeBar': False, 'scrollZoom': True},
                style={'background':'#00FC87','padding-bottom':'2px','padding-left':'2px','height':'100vh'}
            )
        ], className='nine columns'
        ),

    ], className='row'
    ),

], className='ten columns offset-by-one'
)

#---------------------------------------------------------------
# Output of Graph
@app.callback(Output('graph', 'figure'),
              [Input('boro_name', 'value'),
               Input('recycling_type', 'value')])

def update_figure(chosen_boro,chosen_recycling):
    df_sub = df[(df['boro'].isin(chosen_boro)) &
                (df['type'].isin(chosen_recycling))]

    # Create figure
    locations=[go.Scattermapbox(
                    lon = df_sub['longitude'],
                    lat = df_sub['latitude'],
                    mode='markers',
                    marker={'color' : df_sub['color']},
                    unselected={'marker' : {'opacity':1}},
                    selected={'marker' : {'opacity':0.5, 'size':25}},
                    hoverinfo='text',
                    hovertext=df_sub['hov_txt'],
                    customdata=df_sub['website']
    )]

    # Return figure
    return {
        'data': locations,
        'layout': go.Layout(
            uirevision= 'foo', #preserves state of figure/map after callback activated
            clickmode= 'event+select',
            hovermode='closest',
            hoverdistance=2,
            title=dict(text="邊度搵口罩?",font=dict(size=50, color='green')),
            mapbox=dict(
                accesstoken=mapbox_access_token,
                bearing=25,
                style='light',
                center=dict(
                    lat=22.156760,
                    lon=113.558500
                ),
                pitch=40,
                zoom=11.5
            ),
        )
    }
#---------------------------------------------------------------
# callback for Web_link
@app.callback(
    Output('web_link', 'children'),
    [Input('graph', 'clickData')])
def display_click_data(clickData):
    if clickData is None:
        return '點擊想購買口罩的位置,以獲取更多資訊'
    else:
        # print (clickData)
        the_link=clickData['points'][0]['customdata']
        if the_link is None:
            return 'No Website Available'
        else:
            return html.A(the_link, href=the_link, target="_blank")
# #--------------------------------------------------------------
if __name__ == '__main__':
    app.run_server(debug=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [07/Feb/2020 16:13:43] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [07/Feb/2020 16:13:43] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [07/Feb/2020 16:13:43] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [07/Feb/2020 16:13:43] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [07/Feb/2020 16:13:44] "POST /_dash-update-component HTTP/1.1" 200 -
